In [16]:
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import json
import time
from csv import DictWriter
import unidecode

In [2]:
def mois(mois):
    if mois == 'janvier':
        return '01'
    elif mois == 'février':
        return '02'
    elif mois == 'mars':
        return '03'
    elif mois == 'avril':
        return '04'
    elif mois == 'mai':
        return '05'
    elif mois == 'juin':
        return '06'
    elif mois == 'juillet':
        return '07'
    elif mois == 'août':
        return '08'
    elif mois == 'septembre':
        return '09'
    elif mois == 'octobre':
        return '10'
    elif mois == 'novembre':
        return '11'
    elif mois == 'décembre':
        return '12'

In [3]:
#Opening a Chrome browser
nos_deputes = webdriver.Chrome(r'chromedriver.exe')

#Open the page with the list of the national assembly folders
nos_deputes.get('https://www.nosdeputes.fr/deputes')

In [4]:
#Getting all of the folders in the page
tableaux = nos_deputes.find_elements_by_class_name('list_table')

In [5]:
#The web page is divided into several squares containing the different links
#We get all these liks in lst_liens in order to iterate through them
lst_liens = []
for tableau in tableaux:
    for depute in tableau.find_elements_by_tag_name('a'):
        lst_liens.append(depute.get_attribute('href'))

In [22]:
#This count variable is only useful in case the program crashes to avoid going through all the loop again
#As the script only add lines to an existing csv, nothing will ever be rewritten
count = 0

#It iterates through all the links of lst_liens from the "count" variable index
for i in range(count, len(lst_liens)):
    #We wait 3 seconds so that the page has the time to fully charge
    time.sleep(3)
    #Going to the link
    nos_deputes.get(lst_liens[count])
    
    #Getting informations
    nom = nos_deputes.find_element_by_class_name('info_depute').find_element_by_tag_name('h1').text
    
        
    groupe = nos_deputes.find_element_by_class_name('info_depute').find_element_by_tag_name('h2')\
            .find_element_by_tag_name('a').text
    
    #There are a few try/except statement in this notebook.
    #These are used because the page structure is not the same depending on if the deputy is still in activity or not
    try:
        semaines_activ = nos_deputes.find_element_by_class_name('barre_activite')\
                                    .find_elements_by_tag_name('a')[0].text[2:]
        
        presence_commission = nos_deputes.find_element_by_class_name('barre_activite')\
                                    .find_elements_by_tag_name('a')[1].text[2:]
        
        intervention_commission = nos_deputes.find_element_by_class_name('barre_activite')\
                                    .find_elements_by_tag_name('a')[2].text[2:]
        
        pres_hemy = nos_deputes.find_element_by_class_name('barre_activite').find_elements_by_tag_name('a')[3].text[2:]
        
        interv_long_hemy = nos_deputes.find_element_by_class_name('barre_activite')\
                        .find_elements_by_tag_name('a')[4].text[2:]
        
        amend_propos = nos_deputes.find_element_by_class_name('barre_activite').find_elements_by_tag_name('a')[5].text[2:]
        
        rapport_ecr = nos_deputes.find_element_by_class_name('barre_activite').find_elements_by_tag_name('a')[6].text[2:]
        
        propos_loi_ecr = nos_deputes.find_element_by_class_name('barre_activite').find_elements_by_tag_name('a')[7].text[2:]
        
        propos_loi_sign = nos_deputes.find_element_by_class_name('barre_activite').find_elements_by_tag_name('a')[8].text[2:]
            
        quest_ecr = nos_deputes.find_element_by_class_name('barre_activite').find_elements_by_tag_name('a')[9].text[2:]
        
        quest_ora = nos_deputes.find_element_by_class_name('barre_activite').find_elements_by_tag_name('a')[10].text[2:]
        
    except:
        
        semaines_activ = nos_deputes.find_element_by_class_name('barre_activite')\
                                    .find_elements_by_tag_name('li')[0].text[2:]
        
        presence_commission = nos_deputes.find_element_by_class_name('barre_activite')\
                                    .find_elements_by_tag_name('li')[1].text[2:]
        
        intervention_commission = nos_deputes.find_element_by_class_name('barre_activite')\
                                    .find_elements_by_tag_name('li')[2].text[2:]
        
        pres_hemy = nos_deputes.find_element_by_class_name('barre_activite')\
                                    .find_elements_by_tag_name('li')[3].text[2:]
        
        interv_long_hemy = nos_deputes.find_element_by_class_name('barre_activite')\
                                    .find_elements_by_tag_name('li')[4].text[2:]
        
        amend_propos = nos_deputes.find_element_by_class_name('barre_activite')\
                                    .find_elements_by_tag_name('li')[5].text[2:]
        
        rapport_ecr = nos_deputes.find_element_by_class_name('barre_activite')\
                                    .find_elements_by_tag_name('li')[6].text[2:]
        
        propos_loi_ecr = nos_deputes.find_element_by_class_name('barre_activite')\
                                    .find_elements_by_tag_name('li')[7].text[2:]
        
        propos_loi_sign = nos_deputes.find_element_by_class_name('barre_activite')\
                                    .find_elements_by_tag_name('li')[8].text[2:]
            
        quest_ecr = nos_deputes.find_element_by_class_name('barre_activite')\
                                    .find_elements_by_tag_name('li')[9].text[2:]
        
        quest_ora = nos_deputes.find_element_by_class_name('barre_activite')\
                                    .find_elements_by_tag_name('li')[10].text[2:]
        
    #Skipping deputies that didn't have any activity (generally they resigned the day of their nomination)
    if semaines_activ == '0':
        count +=1
        print(count)
        continue
    else:
        pass
    
    #Another condition to make the distinction between active deputies and the others
    if nos_deputes.find_element_by_class_name('boite_depute').find_element_by_tag_name('ul')\
            .find_elements_by_tag_name('li')[0].text[:6] != "Mandat":
        
        fonction_assemblee = nos_deputes.find_element_by_class_name('boite_depute').find_element_by_tag_name('ul')\
            .find_elements_by_tag_name('li')[0].text
        
        parti = nos_deputes.find_element_by_id('b1')\
            .find_elements_by_tag_name('li')[3].text.split(':')[1]
        
        if groupe == 'NI':
            fonction_parti = 'null'
        
        else:
            fonction_parti = nos_deputes.find_element_by_class_name('boite_depute').find_element_by_tag_name('ul')\
            .find_elements_by_tag_name('li')[2].text.split('(')[1][:-1]

        if  nos_deputes.find_element_by_class_name('boite_depute').find_element_by_tag_name('ul')\
            .find_elements_by_tag_name('li')[0].text[:11] != 'Mandat clos':

        
            statut_mandat = "En cours"       
        
        else:
   
            statut_mandat = "Clos"
        
    
        if groupe == 'NI':
            fonction_parti = 'null'
        
        else:
            fonction_parti = nos_deputes.find_element_by_class_name('boite_depute').find_element_by_tag_name('ul')\
            .find_elements_by_tag_name('li')[2].text.split('(')[1][:-1]
            
    else:
        
        fonction_assemblee = 'null'
        
        parti = nos_deputes.find_element_by_id('b1')\
            .find_elements_by_tag_name('li')[2].text.split(':')[1]
        
        if  nos_deputes.find_element_by_class_name('boite_depute').find_element_by_tag_name('ul')\
            .find_elements_by_tag_name('li')[0].text[:11] != 'Mandat clos':
        
            statut_mandat = "En cours"       
        
        else:
        
            statut_mandat = "Clos"
        
    
        if groupe == 'NI':
            fonction_parti = 'NI'
        
        else:
            fonction_parti = nos_deputes.find_element_by_class_name('boite_depute').find_element_by_tag_name('ul')\
            .find_elements_by_tag_name('li')[1].text.split('(')[1][:-1]    
                
        
    profession = nos_deputes.find_element_by_class_name('boite_depute').find_element_by_tag_name('ul')\
            .find_elements_by_tag_name('li')[3].text[13:]
     
    #Some deputies with very little activity don't have a lot of informations about what they did. 
    #We consider them as inactive on everything to make things simpler, as they won't be very useful for us anyway
    try :
        total_propos = nos_deputes.find_element_by_class_name('sorts_amendements')\
            .find_elements_by_tag_name('tr')[1].find_elements_by_tag_name('td')[0].text
        
        total_signes = nos_deputes.find_element_by_class_name('sorts_amendements')\
            .find_elements_by_tag_name('tr')[1].find_elements_by_tag_name('td')[2].text
        
        prop_adopt = nos_deputes.find_element_by_class_name('sorts_amendements')\
            .find_elements_by_tag_name('tr')[2].find_elements_by_tag_name('td')[0].text
        
        sign_adopt = nos_deputes.find_element_by_class_name('sorts_amendements')\
            .find_elements_by_tag_name('tr')[2].find_elements_by_tag_name('td')[2].text
        
        prop_rejet = nos_deputes.find_element_by_class_name('sorts_amendements')\
            .find_elements_by_tag_name('tr')[3].find_elements_by_tag_name('td')[0].text
        
        sign_rejet = nos_deputes.find_element_by_class_name('sorts_amendements')\
            .find_elements_by_tag_name('tr')[3].find_elements_by_tag_name('td')[2].text
        
        prop_tomb = nos_deputes.find_element_by_class_name('sorts_amendements')\
            .find_elements_by_tag_name('tr')[4].find_elements_by_tag_name('td')[0].text
        
        sign_tomb = nos_deputes.find_element_by_class_name('sorts_amendements')\
            .find_elements_by_tag_name('tr')[4].find_elements_by_tag_name('td')[2].text
        
        prop_reti = nos_deputes.find_element_by_class_name('sorts_amendements')\
            .find_elements_by_tag_name('tr')[5].find_elements_by_tag_name('td')[0].text
        
        sign_reti = nos_deputes.find_element_by_class_name('sorts_amendements')\
            .find_elements_by_tag_name('tr')[5].find_elements_by_tag_name('td')[2].text
        
        prop_non_sout = nos_deputes.find_element_by_class_name('sorts_amendements')\
            .find_elements_by_tag_name('tr')[6].find_elements_by_tag_name('td')[0].text
        
        sign_non_sout = nos_deputes.find_element_by_class_name('sorts_amendements')\
                .find_elements_by_tag_name('tr')[6].find_elements_by_tag_name('td')[2].text
        
        prop_irr = nos_deputes.find_element_by_class_name('sorts_amendements')\
            .find_elements_by_tag_name('tr')[7].find_elements_by_tag_name('td')[0].text
        
        sign_irr = nos_deputes.find_element_by_class_name('sorts_amendements')\
            .find_elements_by_tag_name('tr')[7].find_elements_by_tag_name('td')[2].text
        
        prop_en_att = nos_deputes.find_element_by_class_name('sorts_amendements')\
            .find_elements_by_tag_name('tr')[8].find_elements_by_tag_name('td')[0].text
        
        sign_en_att = nos_deputes.find_element_by_class_name('sorts_amendements')\
            .find_elements_by_tag_name('tr')[8].find_elements_by_tag_name('td')[2].text
    except:
        total_propos = total_signes = prop_adopt = sign_adopt = prop_rejet = sign_rejet = prop_tomb = sign_tomb = prop_reti = sign_reti \
        = prop_non_sout = sign_non_sout = prop_irr = sign_irr = prop_en_att = sign_en_att = 'null'
        
    #We save everything in a dictionary, and then as a new line of a csv
    with open('depute_database.csv', 'a+', newline='', encoding="latin-1") as write_obj:
        dict_writer = DictWriter(write_obj, fieldnames=['nom',
                                                                    'semaines_activite', 
                                                                    'presences_commission',  
                                                                    'intervention_commission',
                                                                    'presences_hemycicles',
                                                                    'interventons_longues_hemycicle',
                                                                    'amendements_proposes',
                                                                    'rapports_ecrits',
                                                                    'propositions_loi_ecrites',
                                                                    'propositions_loi_signees',
                                                                    'questions_ecrites', 
                                                                    'questions_orales',
                                                                    'groupe',
                                                                    'parti',
                                                                    'fonction_parti',
                                                                    'profession',
                                                                    'total_propositions',
                                                                    'total_signes',
                                                                    'propositions_adoptees',
                                                                    'signes_adoptes',
                                                                    'propositions_rejetees',
                                                                    'signatures_rejetees',
                                                                    'propositions_tombees',
                                                                    'signatures_tombees',
                                                                    'propositions_retirees',
                                                                    'signatures_retirees',
                                                                    'propositions_non_soutenues',
                                                                    'signatures_non_soutenues',
                                                                    'propositions_irrecevables',
                                                                    'signatures_irrecevables',
                                                                    'propositions_en_attente',
                                                                    'signatures_en_attente',
                                                                    'statut_mandat',
                                                                    'fonction_assemblee'])
            
        final_dict = {'nom' : unidecode.unidecode(nom),
                                            'semaines_activite' : semaines_activ, 
                                            'presences_commission' : presence_commission,  
                                            'intervention_commission' : intervention_commission,
                                            'presences_hemycicles' : pres_hemy,
                                            'interventons_longues_hemycicle' : interv_long_hemy,
                                            'amendements_proposes' : amend_propos,
                                            'rapports_ecrits' : rapport_ecr,
                                            'propositions_loi_ecrites' : propos_loi_ecr,
                                            'propositions_loi_signees' : propos_loi_sign,
                                            'questions_ecrites' : quest_ecr, 
                                            'questions_orales' : quest_ora, 
                                            'parti' : unidecode.unidecode(parti),
                                            'groupe' : unidecode.unidecode(groupe),
                                            'fonction_parti' : unidecode.unidecode(fonction_parti),
                                            'profession' : unidecode.unidecode(profession),
                                            'total_propositions' : total_propos,
                                            'total_signes' : total_signes,
                                            'propositions_adoptees' : prop_adopt,
                                            'signes_adoptes' : sign_adopt,
                                            'propositions_rejetees' : prop_rejet,
                                            'signatures_rejetees' : sign_rejet,
                                            'propositions_tombees' : prop_tomb,
                                            'signatures_tombees' : sign_tomb,
                                            'propositions_retirees': prop_reti,
                                            'signatures_retirees':sign_reti,
                                            'propositions_non_soutenues':prop_reti,
                                            'signatures_non_soutenues': sign_non_sout,
                                            'propositions_irrecevables': prop_irr,
                                            'signatures_irrecevables': sign_irr,
                                            'propositions_en_attente' : prop_en_att,
                                            'signatures_en_attente': sign_en_att,
                                            'statut_mandat': statut_mandat,
                                            'fonction_assemblee' : unidecode.unidecode(fonction_assemblee)}
        
        dict_writer.writerow(final_dict)
    
    #Printing the count allow us to check the progression, but also restart the program from where it was precisely
    count +=1
    print(count)
            


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
